## **0. Environment Setup**




In [1]:
DATASET_PATH = "../data/"

In [2]:
import os
import cv2
import numpy as np
import pickle
import sys
import random

sys.path.append(os.path.abspath('..'))

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from src.preprocessing import extract_pose_vector

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

I0000 00:00:1766796447.170363 12430209 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 90.5), renderer: Apple M1 Pro
I0000 00:00:1766796447.177854 12430209 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 90.5), renderer: Apple M1 Pro


In [3]:
print("DATASET_PATH content:")
!ls "$DATASET_PATH"

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1766796447.194686 12433054 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1766796447.233668 12433054 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1766796447.277340 12433043 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1766796447.294571 12433048 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


DATASET_PATH content:
1         3         5         7         README.md
2         4         6         8


In [9]:
X = []
y = []

video_ext = (".mp4", ".avi", ".mov", ".mkv")

for class_name in sorted(os.listdir(DATASET_PATH)):
    class_path = os.path.join(DATASET_PATH, class_name)
    if not os.path.isdir(class_path):
        continue

    print(class_name)

    for fname in sorted(os.listdir(class_path)):
        if not fname.lower().endswith(video_ext):
            continue

        video_path = os.path.join(class_path, fname)
        print(fname)

        cap = cv2.VideoCapture(video_path)
        frame_id = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            if frame_id % 2 == 0:
                
                # Copy frame for augmentation
                aug_frame = frame.copy()


                # Prevents model from bias to one side
                if random.random() > 0.5:
                    aug_frame = cv2.flip(aug_frame, 1)


                angle = random.uniform(-15, 15)
                h, w = aug_frame.shape[:2]
                M = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1.0)
                aug_frame = cv2.warpAffine(aug_frame, M, (w, h))


                brightness = random.uniform(0.7, 1.3)
                aug_frame = cv2.convertScaleAbs(aug_frame, alpha=brightness, beta=0)
                
                # Process augmented frame
                vec = extract_pose_vector(aug_frame)
                
                if vec is not None:
                    X.append(vec)
                    y.append(class_name)

            frame_id += 1

        cap.release()

X = np.array(X)
y = np.array(y)

print("Total poses:", len(X))
print("Classes:", np.unique(y))


1
1_1.mov
1_2.mov
1_3.mov
1_4.MOV
1_5.mp4
2
2_1.mov
2_2.mov
2_3.mov
2_4.MOV
2_5.mp4
3
3_1.mov
3_2.mov
3_3.mov
3_4.MOV
3_5.mp4
4
4_1.mov
4_2.mov
4_3.mov
4_4.MOV
4_5.mp4
5
5_1.mov
5_2.mov
5_3.mov
5_4.MOV
5_5.mp4
6
6_1.mov
6_2.mov
6_3.mov
6_4.MOV
6_5.mp4
7
7_1.mov
7_2.mov
7_3.mov
7_4.mov
7_5.mp4
8
8_1.mov
8_2.mov
8_3.mov
8_4.mov
8_5.mov
Total poses: 4255
Classes: ['1' '2' '3' '4' '5' '6' '7' '8']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

pca = PCA(n_components=0.98)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)
print(f"PCA: {X_train_scaled.shape[1]} -> {X_train_pca.shape[1]} components")


PCA: 126 -> 26 components (98% variance)


In [8]:
# we've had parameter-picking here but after finding a good one we've settled down.
clf = SVC(C=200, gamma=0.05, kernel='rbf', probability=True)
clf.fit(X_train_pca, y_train)

y_pred = clf.predict(X_test_pca)
print("Quality report:\n")
print(classification_report(y_test, y_pred))
print("Confusion matrix:")
print(confusion_matrix(y_test, y_pred))


Quality report:

              precision    recall  f1-score   support

           1       0.95      0.95      0.95       146
           2       0.98      0.96      0.97        90
           3       0.96      0.97      0.97       129
           4       0.76      1.00      0.86        59
           5       0.97      0.95      0.96       117
           6       0.96      0.95      0.96       137
           7       0.98      0.96      0.97       134
           8       0.96      0.68      0.80        38

    accuracy                           0.95       850
   macro avg       0.94      0.93      0.93       850
weighted avg       0.95      0.95      0.95       850

Confusion matrix:
[[139   1   1   1   0   4   0   0]
 [  0  86   0   4   0   0   0   0]
 [  0   0 125   0   3   0   1   0]
 [  0   0   0  59   0   0   0   0]
 [  1   0   3   1 111   0   1   0]
 [  4   1   0   1   0 130   1   0]
 [  2   0   1   0   0   1 129   1]
 [  0   0   0  12   0   0   0  26]]


In [7]:
model = {
    "clf": clf,
    "scaler": scaler,
    "pca": pca,
    "classes": sorted(list(np.unique(y)))
}

model_path = "../models/model.pkl"
with open(model_path, "wb") as f:
    pickle.dump(model, f)

print("Model saved at:", model_path)


Model saved at: ../models/model.pkl
